In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import warnings
import os
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.datasets import make_classification
import matplotlib 

ModuleNotFoundError: No module named 'matplotlib'

In [414]:
# Ensure you have the NLTK data downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/deeksha_ramakrishna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/deeksha_ramakrishna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [415]:
from nltk.corpus import stopwords
print(stopwords.words('english')[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [416]:
# Define the stop words
stop_words = set(stopwords.words('english'))

In [417]:
warnings.filterwarnings("ignore")

In [418]:
import chardet

with open("data/wcpr_mypersonality.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(result)  # Example: {'encoding': 'Windows-1252', 'confidence': 0.99}


{'encoding': 'Windows-1252', 'confidence': 0.7299572146246162, 'language': ''}


In [419]:
df = pd.read_csv("data/wcpr_mypersonality.csv", sep=',', encoding="windows-1252",  on_bad_lines='skip')

In [420]:
df.head(10)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
5,b7b7764cfa1c523e4e93ab2a79a946c4,www.thejokerblogs.com,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/16/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
6,b7b7764cfa1c523e4e93ab2a79a946c4,"saw a nun zombie, and liked it. Also, *PROPNAM...",2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/27/09 05:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
7,b7b7764cfa1c523e4e93ab2a79a946c4,is in Kentucky. 421 miles into her 1100 mile j...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/18/09 06:34 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
8,b7b7764cfa1c523e4e93ab2a79a946c4,was about to finish a digital painting before ...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/09/09 02:58 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
9,b7b7764cfa1c523e4e93ab2a79a946c4,is celebrating her new haircut by listening to...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/07/09 11:41 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [421]:
df.shape

(9917, 20)

In [422]:
df.columns = df.columns.str.lower()

In [423]:
df.head()

,#authid,status,sext,sneu,sagr,scon,sopn,cext,cneu,cagr,ccon,copn,date,networksize,betweenness,nbetweenness,density,brokerage,nbrokerage,transitivity
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [424]:
print(df.dtypes)

#authid          object
status           object
sext            float64
sneu            float64
sagr            float64
scon            float64
sopn            float64
cext             object
cneu             object
cagr             object
ccon             object
copn             object
date             object
networksize     float64
betweenness     float64
nbetweenness    float64
density         float64
brokerage       float64
nbrokerage      float64
transitivity    float64
dtype: object


In [425]:
df = df.dropna()

In [426]:
df.isnull().any()

#authid         False
status          False
sext            False
sneu            False
sagr            False
scon            False
sopn            False
cext            False
cneu            False
cagr            False
ccon            False
copn            False
date            False
networksize     False
betweenness     False
nbetweenness    False
density         False
brokerage       False
nbrokerage      False
transitivity    False
dtype: bool

In [427]:
df['cneu'] = df['cneu'].replace({'y': 1, 'n': 0})
df['cext'] = df['cext'].replace({'y': 1, 'n': 0})
df['cagr'] = df['cagr'].replace({'y': 1, 'n': 0})
df['ccon'] = df['ccon'].replace({'y': 1, 'n': 0})
df['copn'] = df['copn'].replace({'y': 1, 'n': 0})

In [428]:
df.cneu

0       1
1       1
2       1
3       1
4       1
       ..
9912    0
9913    1
9914    1
9915    1
9916    1
Name: cneu, Length: 9916, dtype: int64

In [431]:
df.dtypes

#authid          object
status           object
sext            float64
sneu            float64
sagr            float64
scon            float64
sopn            float64
cext              int64
cneu              int64
cagr              int64
ccon              int64
copn              int64
date             object
networksize     float64
betweenness     float64
nbetweenness    float64
density         float64
brokerage       float64
nbrokerage      float64
transitivity    float64
dtype: object

In [367]:
#Y = df['cneu']

In [368]:
#Y.head(10)

In [432]:
# drop the unwanted columns from the dataset 
X = df.drop(['#authid'], axis=1)

In [433]:
nlp = spacy.load("en_core_web_sm")

In [434]:
# Function to remove punctuation using spaCy
def remove_punctuation_spacy(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if not token.is_punct])

In [435]:
# Apply to the column
X['status'] = X['status'].apply(remove_punctuation_spacy)

In [436]:
# Remove the digits
X['status'] = X['status'].str.replace(r'\d+', '', regex=True)
X['status'] = X['status'].str.replace(r'/\W/', '', regex=True)


In [437]:
def clean_tokens(tokens):
    return [
        item for item in tokens 
        if len(item) >3 and re.match(r"^[a-zA-Z]+$", item)  # Keep words with 3+ characters and only letters
    ]

In [438]:
import nltk
from nltk.tokenize import word_tokenize

# Download the tokenizer model
nltk.download('punkt_tab')

# Apply tokenization
X['status'] = X['status'].astype(str).apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/deeksha_ramakrishna/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [439]:
X.head()

,status,sext,sneu,sagr,scon,sopn,cext,cneu,cagr,ccon,copn,date,networksize,betweenness,nbetweenness,density,brokerage,nbrokerage,transitivity
0,"[likes, the, sound, of, thunder]",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,"[is, so, sleepy, it, 's, not, even, funny, tha...",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,"[is, sore, and, wants, the, knot, of, muscles,...",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,"[likes, how, the, day, sounds, in, this, new, ...",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,"[is, home, <]",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [440]:
# remove stop words
X['status'] = X['status'].apply(lambda x: [item for item in x if item.lower() not in stop_words])


In [441]:
# remove words that has less than 3 characters and also remove any unwanted punctuations and symbols
X['status'] = X['status'].apply(clean_tokens)

In [442]:
X.head()

,status,sext,sneu,sagr,scon,sopn,cext,cneu,cagr,ccon,copn,date,networksize,betweenness,nbetweenness,density,brokerage,nbrokerage,transitivity
0,"[likes, sound, thunder]",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,"[sleepy, even, funny, sleep]",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,"[sore, wants, knot, muscles, base, neck, stop,...",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,"[likes, sounds, song]",2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,[home],2.65,3.0,3.15,3.25,4.4,0,1,0,0,1,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [443]:
# Define a function to clean the tokenized text
def remove_excessive_repetition_from_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        # Remove words with excessive character repetition
        if not re.search(r'(\w)\1{2,}', token):  # Matches words with character repeated 3+ times
            cleaned_tokens.append(token)  # Keep valid tokens
    return cleaned_tokens

In [444]:
X['status'] = X['status'].apply(remove_excessive_repetition_from_tokens)

In [445]:
X.head

<bound method NDFrame.head of                                                  status  sext  sneu  sagr  \
0                               [likes, sound, thunder]  2.65  3.00  3.15   
1                          [sleepy, even, funny, sleep]  2.65  3.00  3.15   
2     [sore, wants, knot, muscles, base, neck, stop,...  2.65  3.00  3.15   
3                                 [likes, sounds, song]  2.65  3.00  3.15   
4                                                [home]  2.65  3.00  3.15   
...                                                 ...   ...   ...   ...   
9912                       [little, things, give, away]  2.15  2.15  4.10   
9913                                [wishing, Saturday]  4.05  3.35  3.80   
9914                                   [studying, hard]  4.05  3.35  3.80   
9915                                    [snipers, head]  1.40  4.05  3.30   
9916  [Last, night, amazing, PROPNAME, PROPNAME, LAS...  4.25  3.00  3.25   

      scon  sopn  cext  cneu  cagr  ccon  cop

In [446]:
# Define a function to join tokenized lists into strings
def join_tokens_to_sentence(tokens):
    return ' '.join(tokens)

In [447]:
X['status'] = X['status'].apply(join_tokens_to_sentence)

In [448]:
vectorizer = CountVectorizer(ngram_range=(1,1))
X_grams = vectorizer.fit_transform(X['status'])
num_features = len(vectorizer.vocabulary_)

In [449]:
vectorizer.get_feature_names_out()

array(['aaliyah', 'aamuks', 'aarp', ..., 'zulu', 'zumba', 'zynga'],
      dtype=object)

In [450]:
print(num_features)

13147


In [451]:
print(X_grams.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [452]:
from sklearn.decomposition import PCA

# Reduce to 10 principal components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_grams.toarray())


In [453]:
X_pca

array([[-0.14145411, -0.11157354],
       [-0.12807691, -0.05618625],
       [-0.14462692, -0.11990931],
       ...,
       [-0.14062053, -0.09837304],
       [-0.14186783, -0.11494777],
       [ 1.92357067,  0.0187481 ]])

In [454]:
pca_df = pd.DataFrame(X_pca, columns=['pca1', 'pca2'])

In [455]:
pca_df.head()

,pca1,pca2
0,-0.141454,-0.111574
1,-0.128077,-0.056186
2,-0.144627,-0.119909
3,-0.139868,-0.085465
4,-0.131026,-0.077217


In [456]:
X = pd.concat([X.reset_index(drop=True), pca_df], axis=1)

In [457]:
X.columns

Index(['status', 'sext', 'sneu', 'sagr', 'scon', 'sopn', 'cext', 'cneu',
       'cagr', 'ccon', 'copn', 'date', 'networksize', 'betweenness',
       'nbetweenness', 'density', 'brokerage', 'nbrokerage', 'transitivity',
       'pca1', 'pca2'],
      dtype='object')

In [458]:
X.head

<bound method NDFrame.head of                                                  status  sext  sneu  sagr  \
0                                   likes sound thunder  2.65  3.00  3.15   
1                               sleepy even funny sleep  2.65  3.00  3.15   
2     sore wants knot muscles base neck stop hurting...  2.65  3.00  3.15   
3                                     likes sounds song  2.65  3.00  3.15   
4                                                  home  2.65  3.00  3.15   
...                                                 ...   ...   ...   ...   
9911                            little things give away  2.15  2.15  4.10   
9912                                   wishing Saturday  4.05  3.35  3.80   
9913                                      studying hard  4.05  3.35  3.80   
9914                                       snipers head  1.40  4.05  3.30   
9915  Last night amazing PROPNAME PROPNAME LAST perf...  4.25  3.00  3.25   

      scon  sopn  cext  cneu  cagr  ccon  ...

In [459]:
X.isnull().any()

status          False
sext            False
sneu            False
sagr            False
scon            False
sopn            False
cext            False
cneu            False
cagr            False
ccon            False
copn            False
date            False
networksize     False
betweenness     False
nbetweenness    False
density         False
brokerage       False
nbrokerage      False
transitivity    False
pca1            False
pca2            False
dtype: bool

In [460]:
X = X.dropna()

In [461]:
X.shape

(9916, 21)

In [462]:
X.isnull().any()

status          False
sext            False
sneu            False
sagr            False
scon            False
sopn            False
cext            False
cneu            False
cagr            False
ccon            False
copn            False
date            False
networksize     False
betweenness     False
nbetweenness    False
density         False
brokerage       False
nbrokerage      False
transitivity    False
pca1            False
pca2            False
dtype: bool

In [463]:
Y = X['cneu']

In [464]:
Y.head

<bound method NDFrame.head of 0       1
1       1
2       1
3       1
4       1
       ..
9911    0
9912    1
9913    1
9914    1
9915    1
Name: cneu, Length: 9916, dtype: int64>

In [465]:
Y.shape

(9916,)

In [ ]:
X = X.drop(['cneu'], axis =1)

In [471]:
X = X.drop(['status', 'date'], axis =1)

In [472]:
X.shape

(9916, 18)

In [473]:
X.columns

Index(['sext', 'sneu', 'sagr', 'scon', 'sopn', 'cext', 'cagr', 'ccon', 'copn',
       'networksize', 'betweenness', 'nbetweenness', 'density', 'brokerage',
       'nbrokerage', 'transitivity', 'pca1', 'pca2'],
      dtype='object')

In [474]:
X.head(1)

,sext,sneu,sagr,scon,sopn,cext,cagr,ccon,copn,networksize,betweenness,nbetweenness,density,brokerage,nbrokerage,transitivity,pca1,pca2
0,2.65,3.0,3.15,3.25,4.4,0,0,0,1,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1,-0.141454,-0.111574


In [475]:
# Split the datafram in to train and test with random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [476]:
X_train.shape

(6941, 18)

In [478]:
X_train.isnull()

,sext,sneu,sagr,scon,sopn,cext,cagr,ccon,copn,networksize,betweenness,nbetweenness,density,brokerage,nbrokerage,transitivity,pca1,pca2
1420,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5173,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4827,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8737,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7401,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5191,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5390,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
860,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [479]:
X_test.shape

(2975, 18)

In [480]:
clf = LogisticRegression(random_state=42)

In [481]:
clf.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [484]:
_ = ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test)

ImportError: ConfusionMatrixDisplay.from_estimator requires matplotlib. You can install matplotlib with `pip install matplotlib`